In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# 读取数据
file_path = 'Submited version.csv'  # 修改为您的文件路径
data = pd.read_csv(file_path)

# 选择加拿大的数据
canada_data = data[data['Economies'] == 'Canada'][['Year', 'Global Innovation Index']].dropna()

# 设置年份为索引
canada_data.set_index('Year', inplace=True)

# 数据归一化（可选）
scaler = MinMaxScaler()
canada_data['Global Innovation Index'] = scaler.fit_transform(canada_data[['Global Innovation Index']])


In [ ]:
import statsmodels.api as sm

# 创建滞后特征
canada_data['Lag_1'] = canada_data['Global Innovation Index'].shift(1)
canada_data = canada_data.dropna()

# 定义自变量 (Lag_1) 和因变量 (Global Innovation Index)
X = canada_data[['Lag_1']]
y = canada_data['Global Innovation Index']

# 添加常数项 (Intercept)
X = sm.add_constant(X)

# 拟合 GLM 模型
glm_model = sm.GLM(y, X, family=sm.families.Gaussian())
glm_results = glm_model.fit()

# 打印模型摘要
print(glm_results.summary())


In [ ]:
# 使用模型进行预测
canada_data['Predicted'] = glm_results.predict(X)

# 可视化实际值与预测值
plt.figure(figsize=(10, 6))
plt.plot(canada_data.index, canada_data['Global Innovation Index'], label='Actual', marker='o')
plt.plot(canada_data.index, canada_data['Predicted'], label='Predicted', marker='x')
plt.title('Time Series Prediction using GLM')
plt.xlabel('Year')
plt.ylabel('Normalized Global Innovation Index')
plt.legend()
plt.grid(True)
plt.show()
